# Imports

In [1]:
import pandas as pd
import numpy  as np
import math

import matplotlib.pyplot as     plt
import seaborn           as     sns
from matplotlib.gridspec import GridSpec

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers    import SentenceTransformer

## Helpers

# Data Load

In [2]:
df = pd.read_csv('Total_clientes_atuais.csv')
df = df.iloc[:,:-2]

# selecionando cols
cols = [
    'CNPJ','RAZÃO SOCIAL','NAT. JURIDICA','QUALIFICAÇÃO DO RESPONSÁVEL','CAPITAL SOCIAL','PORTE DA EMPRESA',
    'NOME FANTASIA','DT. SIT. CAD.','DT. INI. ATIV.','CNAE FISCAL PRINCIPAL','CNAE FISCAL SECUNDÁRIA'
]
df = df[cols].drop_duplicates().copy()
df

,CNPJ,RAZÃO SOCIAL,NAT. JURIDICA,QUALIFICAÇÃO DO RESPONSÁVEL,CAPITAL SOCIAL,PORTE DA EMPRESA,NOME FANTASIA,DT. SIT. CAD.,DT. INI. ATIV.,CNAE FISCAL PRINCIPAL,CNAE FISCAL SECUNDÁRIA
0,43.016.722/0001-47,ALL SOCKS CWB COMERCIO DE MEIAS LTDA,Sociedade Empresária Limitada,Sócio-Administrador,"40.000,00",MICRO EMPRESA,ALL SOCKS CWB,8/6/2021,8/6/2021,4781400,NaN
1,06.246.280/0001-07,ATRAENTE LINGERIE LTDA,Sociedade Empresária Limitada,Sócio-Administrador,"10.000,00",MICRO EMPRESA,ATRAENTE LINGERIE LTDA,NaN,5/10/2004,4781400,NaN
2,05.080.448/0001-86,BAYER FLUTUARE LTDA,Sociedade Empresária Limitada,Sócio-Administrador,"0,00",MICRO EMPRESA,CHOCOLE,4/9/2004,5/29/2002,4781400,"4789001,4713002"
3,98.514.284/0001-39,BELLONY SANTA BROGLIO GEARRAH & CIA LTDA,Sociedade Empresária Limitada,Sócio-Administrador,"85.000,00",MICRO EMPRESA,LOJA MIRIAM,7/29/2001,9/15/1966,4755501,"4755503,4755502,4781400"
4,34.031.912/0001-42,BOT & BOT COMERCIO VAREJISTA DO VESTUARIO LTDA,Sociedade Empresária Limitada,Sócio-Administrador,"100.000,00",MICRO EMPRESA,BOT & BOT COMERCIO VAREJISTA DO VESTUARIO LTDA,6/25/2019,6/25/2019,4781400,"4772500,4782201,4782202,4783101,4783102,478900..."
...,...,...,...,...,...,...,...,...,...,...,...
96,07.272.256/0001-05,TOPA CONFECCOES LTDA,Sociedade Empresária Limitada,Administrador,"50.000,00",DEMAIS,TOPA CONFECCOES LTDA,3/4/2005,3/4/2005,4781400,1351100
97,28.017.097/0001-64,V. BALABUCH NEUMANN - COMERCIO DE ROUPAS,Empresário (Individual),Empresário,"35.000,00",MICRO EMPRESA,INTIMAS LINGERIE,6/22/2017,6/22/2017,4781400,NaN
98,01.324.712/0001-29,V F DO ROSARIO & CIA LTDA,Sociedade Empresária Limitada,Sócio-Administrador,"100.000,00",MICRO EMPRESA,VANIA MODAS,11/3/2005,5/30/1996,4781400,NaN
99,15.045.507/0001-39,VK COMERCIO DE CONFECCOES LTDA,Sociedade Empresária Limitada,Sócio-Administrador,"150.000,00",DEMAIS,VK COMERCIO DE CONFECCOES LTDA,2/6/2012,2/6/2012,4781400,"4755503,4759899,4772500,4782201"


In [3]:
# carregando cnae
df_cnae         = pd.read_csv('CNAE_Subclasses_2_3_Estrutura_Detalhada.csv', header=3, dtype='object')
df_cnae         = df_cnae.replace(' ', np.nan)
df_cnae.columns = ['secao','divisao','grupo','classe','subclasse','desc']
df_cnae

,secao,divisao,grupo,classe,subclasse,desc
0,A,NaN,NaN,NaN,NaN,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
1,NaN,01,NaN,NaN,NaN,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS"
2,NaN,NaN,01.1,NaN,NaN,Produção de lavouras temporárias
3,NaN,NaN,NaN,01.11-3,NaN,Cultivo de cereais
4,NaN,NaN,NaN,NaN,0111-3/01,Cultivo de arroz
...,...,...,...,...,...,...
2392,NaN,NaN,NaN,NaN,9700-5/00,Serviços domésticos
2393,U,NaN,NaN,NaN,NaN,ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕE...
2394,NaN,99,NaN,NaN,NaN,ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕE...
2395,NaN,NaN,99.0,NaN,NaN,Organismos internacionais e outras instituiçõe...


In [4]:
df_secao = df_cnae[['secao','desc']].dropna().reset_index(drop=True).rename(columns={'desc':'secao_desc'}).copy()
df_secao.head()

,secao,secao_desc
0,A,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
1,B,INDÚSTRIAS EXTRATIVAS
2,C,INDÚSTRIAS DE TRANSFORMAÇÃO
3,D,ELETRICIDADE E GÁS
4,E,"ÁGUA, ESGOTO, ATIVIDADES DE GESTÃO DE RESÍDUOS..."


In [5]:
df_cnae['secao'] = df_cnae['secao'].ffill()
df_cnae

,secao,divisao,grupo,classe,subclasse,desc
0,A,NaN,NaN,NaN,NaN,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
1,A,01,NaN,NaN,NaN,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS"
2,A,NaN,01.1,NaN,NaN,Produção de lavouras temporárias
3,A,NaN,NaN,01.11-3,NaN,Cultivo de cereais
4,A,NaN,NaN,NaN,0111-3/01,Cultivo de arroz
...,...,...,...,...,...,...
2392,T,NaN,NaN,NaN,9700-5/00,Serviços domésticos
2393,U,NaN,NaN,NaN,NaN,ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕE...
2394,U,99,NaN,NaN,NaN,ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕE...
2395,U,NaN,99.0,NaN,NaN,Organismos internacionais e outras instituiçõe...


In [6]:
# df parcial secao -> divisao
df_secao_divisao = df_cnae[['secao','divisao','desc']].dropna().reset_index(drop=True).rename(columns={'desc':'divisao_desc'})
df_secao_divisao.head()

,secao,divisao,divisao_desc
0,A,01,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS"
1,A,02,PRODUÇÃO FLORESTAL
2,A,03,PESCA E AQÜICULTURA
3,B,05,EXTRAÇÃO DE CARVÃO MINERAL
4,B,06,EXTRAÇÃO DE PETRÓLEO E GÁS NATURAL


In [7]:
# df parcial divisao -> grupo
df_aux            = df_cnae[['divisao','grupo','desc']].iloc[1:].copy()
df_aux['divisao'] = df_aux['divisao'].ffill()
df_divisao_grupo  = df_aux.dropna().reset_index(drop=True).rename(columns={'desc':'grupo_desc'})
df_divisao_grupo.head()

,divisao,grupo,grupo_desc
0,01,01.1,Produção de lavouras temporárias
1,01,01.2,Horticultura e floricultura
2,01,01.3,Produção de lavouras permanentes
3,01,01.4,Produção de sementes e mudas certificadas
4,01,01.5,Pecuária


In [8]:
# df parcial grupo -> classe
df_aux          = df_cnae[['grupo','classe','desc']].iloc[2:].copy()
df_aux['grupo'] = df_aux['grupo'].ffill()
df_grupo_classe  = df_aux.dropna().reset_index(drop=True).rename(columns={'desc':'classe_desc'})
df_grupo_classe.head()

,grupo,classe,classe_desc
0,01.1,01.11-3,Cultivo de cereais
1,01.1,01.12-1,Cultivo de algodão herbáceo e de outras fibras...
2,01.1,01.13-0,Cultivo de cana-de-açúcar
3,01.1,01.14-8,Cultivo de fumo
4,01.1,01.15-6,Cultivo de soja


In [9]:
# df parcial clasee -> subclasse
df_aux                      = df_cnae[['classe','subclasse','desc']].iloc[2:].copy()
df_aux['classe']            = df_aux['classe'].ffill()
df_classe_subclasse         = df_aux.dropna().reset_index(drop=True).rename(columns={'desc':'subclasse_desc'})
df_classe_subclasse['cnae'] = df_classe_subclasse['subclasse'].apply(lambda x: x.replace('-','').replace('/',''))
df_classe_subclasse         = df_classe_subclasse[['classe','subclasse','cnae','subclasse_desc']]
df_classe_subclasse.head()

,classe,subclasse,cnae,subclasse_desc
0,01.11-3,0111-3/01,0111301,Cultivo de arroz
1,01.11-3,0111-3/02,0111302,Cultivo de milho
2,01.11-3,0111-3/03,0111303,Cultivo de trigo
3,01.11-3,0111-3/99,0111399,Cultivo de outros cereais não especificados an...
4,01.12-1,0112-1/01,0112101,Cultivo de algodão herbáceo


In [10]:
df_aux = df_classe_subclasse.merge(df_grupo_classe)
df_aux = df_aux.merge(df_divisao_grupo)
df_aux = df_aux.merge(df_secao_divisao)
df_aux = df_aux.merge(df_secao)

df_cnae_full = df_aux[['secao','divisao','grupo','classe','subclasse','cnae','secao_desc','divisao_desc','grupo_desc','classe_desc','subclasse_desc']].copy()
df_cnae_full

,secao,divisao,grupo,classe,subclasse,cnae,secao_desc,divisao_desc,grupo_desc,classe_desc,subclasse_desc
0,A,01,01.1,01.11-3,0111-3/01,0111301,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...","AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS",Produção de lavouras temporárias,Cultivo de cereais,Cultivo de arroz
1,A,01,01.1,01.11-3,0111-3/02,0111302,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...","AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS",Produção de lavouras temporárias,Cultivo de cereais,Cultivo de milho
2,A,01,01.1,01.11-3,0111-3/03,0111303,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...","AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS",Produção de lavouras temporárias,Cultivo de cereais,Cultivo de trigo
3,A,01,01.1,01.11-3,0111-3/99,0111399,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...","AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS",Produção de lavouras temporárias,Cultivo de cereais,Cultivo de outros cereais não especificados an...
4,A,01,01.1,01.12-1,0112-1/01,0112101,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...","AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS",Produção de lavouras temporárias,Cultivo de algodão herbáceo e de outras fibras...,Cultivo de algodão herbáceo
...,...,...,...,...,...,...,...,...,...,...,...
1326,S,96,96.0,96.09-2,9609-2/06,9609206,OUTRAS ATIVIDADES DE SERVIÇOS,OUTRAS ATIVIDADES DE SERVIÇOS PESSOAIS,Outras atividades de serviços pessoais,Atividades de serviços pessoais não especifica...,Serviços de tatuagem e colocação de piercing
1327,S,96,96.0,96.09-2,9609-2/07,9609207,OUTRAS ATIVIDADES DE SERVIÇOS,OUTRAS ATIVIDADES DE SERVIÇOS PESSOAIS,Outras atividades de serviços pessoais,Atividades de serviços pessoais não especifica...,Alojamento de animais domésticos
1328,S,96,96.0,96.09-2,9609-2/08,9609208,OUTRAS ATIVIDADES DE SERVIÇOS,OUTRAS ATIVIDADES DE SERVIÇOS PESSOAIS,Outras atividades de serviços pessoais,Atividades de serviços pessoais não especifica...,Higiene e embelezamento de animais domésticos
1329,S,96,96.0,96.09-2,9609-2/99,9609299,OUTRAS ATIVIDADES DE SERVIÇOS,OUTRAS ATIVIDADES DE SERVIÇOS PESSOAIS,Outras atividades de serviços pessoais,Atividades de serviços pessoais não especifica...,Outras atividades de serviços pessoais não esp...


In [11]:
# transformando tipo da coluna cnae principal
df['CNAE FISCAL PRINCIPAL'] = df['CNAE FISCAL PRINCIPAL'].astype(str)

In [12]:
df_final = df.merge(df_cnae_full, how='left', left_on='CNAE FISCAL PRINCIPAL', right_on='cnae')
df_final = df_final[[
    'CNPJ', 'RAZÃO SOCIAL', 'NOME FANTASIA', 'NAT. JURIDICA',
    'PORTE DA EMPRESA', 'CNAE FISCAL PRINCIPAL', 
    'CNAE FISCAL SECUNDÁRIA', 'secao', 'divisao', 'grupo',
    'classe', 'subclasse', 'secao_desc', 'divisao_desc', 
    'grupo_desc', 'classe_desc', 'subclasse_desc'
]]
# removi 'cnae', pq é o mesmo que 'CNAE FISCAL PRINCIPAL'
# removi datas, qualificacao do responsavel e capital social, pois não serao usadas na primeira fase da solucao
# 
df_final.columns = [
    'cnpj', 'razao_social', 'nome_fantasia', 'nat_juridica',
    'porte', 'cnae_principal', 'cnae_secundaria', 'secao',
    'divisao', 'grupo', 'classe', 'subclasse', 'secao_desc',
    'divisao_desc', 'grupo_desc', 'classe_desc', 'subclasse_desc'
]
df_final.head().T

,0,1,2,3,4
cnpj,43.016.722/0001-47,06.246.280/0001-07,05.080.448/0001-86,98.514.284/0001-39,34.031.912/0001-42
razao_social,ALL SOCKS CWB COMERCIO DE MEIAS LTDA,ATRAENTE LINGERIE LTDA,BAYER FLUTUARE LTDA,BELLONY SANTA BROGLIO GEARRAH & CIA LTDA,BOT & BOT COMERCIO VAREJISTA DO VESTUARIO LTDA
nome_fantasia,ALL SOCKS CWB,ATRAENTE LINGERIE LTDA,CHOCOLE,LOJA MIRIAM,BOT & BOT COMERCIO VAREJISTA DO VESTUARIO LTDA
nat_juridica,Sociedade Empresária Limitada,Sociedade Empresária Limitada,Sociedade Empresária Limitada,Sociedade Empresária Limitada,Sociedade Empresária Limitada
porte,MICRO EMPRESA,MICRO EMPRESA,MICRO EMPRESA,MICRO EMPRESA,MICRO EMPRESA
cnae_principal,4781400,4781400,4781400,4755501,4781400
cnae_secundaria,NaN,NaN,"4789001,4713002","4755503,4755502,4781400","4772500,4782201,4782202,4783101,4783102,478900..."
secao,G,G,G,G,G
divisao,47,47,47,47,47
grupo,47.8,47.8,47.8,47.5,47.8


In [13]:
# fazendo copia para manipular cnae secundario
df_aux = df_final.copy()
df_aux['cnae_secundaria'] = df_aux['cnae_secundaria'].apply(lambda x: [] if 'nan' in str(x).split(',') else str(x).split(','))
df_aux

,cnpj,razao_social,nome_fantasia,nat_juridica,porte,cnae_principal,cnae_secundaria,secao,divisao,grupo,classe,subclasse,secao_desc,divisao_desc,grupo_desc,classe_desc,subclasse_desc
0,43.016.722/0001-47,ALL SOCKS CWB COMERCIO DE MEIAS LTDA,ALL SOCKS CWB,Sociedade Empresária Limitada,MICRO EMPRESA,4781400,[],G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
1,06.246.280/0001-07,ATRAENTE LINGERIE LTDA,ATRAENTE LINGERIE LTDA,Sociedade Empresária Limitada,MICRO EMPRESA,4781400,[],G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
2,05.080.448/0001-86,BAYER FLUTUARE LTDA,CHOCOLE,Sociedade Empresária Limitada,MICRO EMPRESA,4781400,"[4789001, 4713002]",G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
3,98.514.284/0001-39,BELLONY SANTA BROGLIO GEARRAH & CIA LTDA,LOJA MIRIAM,Sociedade Empresária Limitada,MICRO EMPRESA,4755501,"[4755503, 4755502, 4781400]",G,47,47.5,47.55-5,4755-5/01,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de equipamentos de informát...,Comércio varejista especializado de tecidos e ...,Comércio varejista de tecidos
4,34.031.912/0001-42,BOT & BOT COMERCIO VAREJISTA DO VESTUARIO LTDA,BOT & BOT COMERCIO VAREJISTA DO VESTUARIO LTDA,Sociedade Empresária Limitada,MICRO EMPRESA,4781400,"[4772500, 4782201, 4782202, 4783101, 4783102, ...",G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,07.272.256/0001-05,TOPA CONFECCOES LTDA,TOPA CONFECCOES LTDA,Sociedade Empresária Limitada,DEMAIS,4781400,[1351100],G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
85,28.017.097/0001-64,V. BALABUCH NEUMANN - COMERCIO DE ROUPAS,INTIMAS LINGERIE,Empresário (Individual),MICRO EMPRESA,4781400,[],G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
86,01.324.712/0001-29,V F DO ROSARIO & CIA LTDA,VANIA MODAS,Sociedade Empresária Limitada,MICRO EMPRESA,4781400,[],G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
87,15.045.507/0001-39,VK COMERCIO DE CONFECCOES LTDA,VK COMERCIO DE CONFECCOES LTDA,Sociedade Empresária Limitada,DEMAIS,4781400,"[4755503, 4759899, 4772500, 4782201]",G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...


In [25]:
df_aux.to_csv('pre-processed-data.csv', index=False)

In [14]:
df_cnae_set = df_aux[['cnae_principal','cnae_secundaria']].groupby('cnae_principal').sum().reset_index().copy()
df_cnae_set['cnae_secundaria'] = df_cnae_set['cnae_secundaria'].apply(lambda x: list(set(x)))
df_cnae_set

,cnae_principal,cnae_secundaria
0,1411801,"[4782202, 4781400, 4755503, 1412601, 1411802]"
1,1412601,"[4781400, 4755501, 4772500, 4782201, 1411801]"
2,4744099,"[4744001, 4742300, 4741500, 4781400, 4755503]"
3,4754701,[]
4,4755501,"[4755502, 4649405, 4781400, 4759801, 7319099, ..."
5,4755503,"[4755502, 4789099, 4752100, 4781400, 4753900, ..."
6,4781400,"[4782202, 4753900, 6204000, 4789009, 4744099, ..."
7,4782201,[]
8,8211300,[]
9,9602501,"[4781400, 9602502]"


# Fase 1: Buscando CNAEs Secundários de Empresas de Mesmo CNAE Principal

Dado o CNAE de uma empresa, o algoritmo faz uma busca de todas as outras empresas que possuem o mesmo CNAE e que também possuem CNAE Secundário para usá-las como sugestão.

In [15]:
def buscar_recomendacao(cnae):
    mask = df_cnae_set['cnae_principal'].apply(lambda x: cnae in x)
    return 'Não há recomendações' if len(df_cnae_set.loc[mask, 'cnae_secundaria'].values[0])==0 else df_cnae_set.loc[mask, 'cnae_secundaria'].values[0]

In [16]:
df_sem_cnae_secundaria = df_final[df_final['cnae_secundaria'].isna()].copy()
cnaes_principais_sem_secundarias = df_final.loc[df_final['cnae_secundaria'].isna(), 'cnae_principal'].unique()
cnaes_principais_sem_secundarias

array(['4781400', '4755503', '4782201', '4754701', '4755501', '8211300'],
      dtype=object)

In [17]:
cnae_primario = '4755503'

recomendacao = buscar_recomendacao(cnae_primario)

print("Recomendação:", recomendacao)

Recomendação: ['4755502', '4789099', '4752100', '4781400', '4753900', '4772500']


In [19]:
df_final.cnae_principal.value_counts()

cnae_principal
4781400    72
4755503     5
4755501     3
4782201     2
1411801     2
1412601     1
9602501     1
4744099     1
4754701     1
8211300     1
Name: count, dtype: int64

In [20]:
top_k = 5
cnpj  = input('Digite o CNPJ do Cliente:').strip()
print()

print("- Cliente:")
df_cliente   = df_final[df_final['cnpj']==cnpj].copy()
cliente_dict = {}
for i in ['cnpj', 'razao_social', 'nome_fantasia', 'cnae_principal', 'cnae_secundaria']:
    if i!='cnae_secundaria':
        cliente_dict[i] = df_cliente[i].values[0]
    else:
        cliente_dict[i] = df_cliente[i].values[0].split(',') if type(df_cliente[i].values[0])==str else []
display(cliente_dict)
print()



print("- Recomendações:")
current_cnae         = df_cliente['cnae_principal'].values[0]
# current_razao_social = df_cliente['razao_social'].values[0]
razao_social_todos = df_final.loc[df_final['cnae_principal']==current_cnae, 'razao_social'].values
df_rec               = df_final[(df_final['cnae_principal']==current_cnae) & (df_final['cnpj']!=cnpj)].copy()

##### TO DO: calcular similaridade entre nomes
# selecionando modelo pre-treinado de acordo com a seguinte documentação: https://sbert.net/docs/pretrained_models.html
model              = SentenceTransformer('distiluse-base-multilingual-cased-v1')

client_embeddings      = model.encode(razao_social_todos)
df_rec['similaridade'] = cosine_similarity(client_embeddings)[1:,0]
df_rec = df_rec.sort_values(by='similaridade', ascending=False)

r = {}
for count, val in enumerate(df_rec['cnpj'].values):
    df_cnpj_current = df_rec[df_rec['cnpj']==val]
    r[count] = {
        'cnpj': val,
        'razao_social': df_cnpj_current['razao_social'].values[0],
        'nome_fantasia': df_cnpj_current['nome_fantasia'].values[0],
        'cnae_principal': df_cnpj_current['cnae_principal'].values[0],
        'cnae_secundaria': df_cnpj_current['cnae_secundaria'].values[0].split(',') if type(df_cnpj_current['cnae_secundaria'].values[0])==str else [],
        'similaridade': df_cnpj_current['similaridade'].values[0]
    }
    if count==top_k-1:
        break
display(r)

Digite o CNPJ do Cliente: 43.016.722/0001-47



- Cliente:


{'cnpj': '43.016.722/0001-47',
 'razao_social': 'ALL SOCKS CWB COMERCIO DE MEIAS LTDA',
 'nome_fantasia': 'ALL SOCKS CWB',
 'cnae_principal': '4781400',
 'cnae_secundaria': []}


- Recomendações:


{0: {'cnpj': '44.999.259/0001-63',
  'razao_social': 'M S M KUCINSKI COMERCIO DE CONFECCOES LTDA',
  'nome_fantasia': 'M S M KUCINSKI COMERCIO DE CONFECCOES LTDA',
  'cnae_principal': '4781400',
  'cnae_secundaria': [],
  'similaridade': 0.7663462},
 1: {'cnpj': '77.988.889/0001-91',
  'razao_social': 'TUDO BOM COMERCIO DE CONFECCOES LTDA',
  'nome_fantasia': 'TUDO BOM COMERCIO DE CONFECCOES LTDA',
  'cnae_principal': '4781400',
  'cnae_secundaria': [],
  'similaridade': 0.7092587},
 2: {'cnpj': '84.856.962/0001-37',
  'razao_social': 'ENAMORATTA COMERCIO DE LINGERIE LTDA',
  'nome_fantasia': 'ENAMORATTA COMERCIO DE LINGERIE LTDA',
  'cnae_principal': '4781400',
  'cnae_secundaria': [],
  'similaridade': 0.67626154},
 3: {'cnpj': '15.045.507/0001-39',
  'razao_social': 'VK COMERCIO DE CONFECCOES LTDA',
  'nome_fantasia': 'VK COMERCIO DE CONFECCOES LTDA',
  'cnae_principal': '4781400',
  'cnae_secundaria': ['4755503', '4759899', '4772500', '4782201'],
  'similaridade': 0.6709931},
 4: {

In [22]:
current_cnae = df_cliente['cnae_principal'].values[0]
df_rec = df_final[(df_final['cnae_principal']==current_cnae) & (df_final['cnpj']!=cnpj)].copy()
df_rec



,cnpj,razao_social,nome_fantasia,nat_juridica,porte,cnae_principal,cnae_secundaria,secao,divisao,grupo,classe,subclasse,secao_desc,divisao_desc,grupo_desc,classe_desc,subclasse_desc
1,06.246.280/0001-07,ATRAENTE LINGERIE LTDA,ATRAENTE LINGERIE LTDA,Sociedade Empresária Limitada,MICRO EMPRESA,4781400,NaN,G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
2,05.080.448/0001-86,BAYER FLUTUARE LTDA,CHOCOLE,Sociedade Empresária Limitada,MICRO EMPRESA,4781400,"4789001,4713002",G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
4,34.031.912/0001-42,BOT & BOT COMERCIO VAREJISTA DO VESTUARIO LTDA,BOT & BOT COMERCIO VAREJISTA DO VESTUARIO LTDA,Sociedade Empresária Limitada,MICRO EMPRESA,4781400,"4772500,4782201,4782202,4783101,4783102,478900...",G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
6,00.143.478/0002-51,COMERCIO DE CONFECCOES STATION LTDA,COMERCIO DE CONFECCOES STATION LTDA,Sociedade Empresária Limitada,MICRO EMPRESA,4781400,NaN,G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
7,00.143.478/0001-70,COMERCIO DE CONFECCOES STATION LTDA,STATION MODAS,Sociedade Empresária Limitada,MICRO EMPRESA,4781400,NaN,G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,05.909.100/0001-59,SILVIA WILBERT,DESEJOS,Empresário (Individual),MICRO EMPRESA,4781400,NaN,G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
84,07.272.256/0001-05,TOPA CONFECCOES LTDA,TOPA CONFECCOES LTDA,Sociedade Empresária Limitada,DEMAIS,4781400,1351100,G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
85,28.017.097/0001-64,V. BALABUCH NEUMANN - COMERCIO DE ROUPAS,INTIMAS LINGERIE,Empresário (Individual),MICRO EMPRESA,4781400,NaN,G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...
86,01.324.712/0001-29,V F DO ROSARIO & CIA LTDA,VANIA MODAS,Sociedade Empresária Limitada,MICRO EMPRESA,4781400,NaN,G,47,47.8,47.81-4,4781-4/00,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,COMÉRCIO VAREJISTA,Comércio varejista de produtos novos não espec...,Comércio varejista de artigos do vestuário e a...,Comércio varejista de artigos do vestuário e a...


# TO DO

- [ ] Dado um CNAE, sugerir lista de CNAE Secundario ainda não utilizado
- [ ] Ordenar sugestões com base na similaridade entre nomes das empresas

In [143]:
# def get_full_cnpj(x):
#     weights = [5, 6, 7, 8, 9, 2, 3, 4, 5, 6, 7, 8, 9]
    
#     soma_1 = sum([int(x[i]) * weights[i+1] for i in range(12)])
#     dv1 = soma_1 % 11
#     dv1 = str(0) if dv1 > 9 else str(dv1)
    
#     x = x + dv1
    
#     soma_2 = sum([int(x[i]) * weights[i] for i in range(13)])
#     dv2 = soma_2 % 11
#     dv2 = str(0) if dv2 > 9 else str(dv2)
    
#     x = x + dv2
    
#     cnpj = x[:2] + '.' + x[2:5] + '.' + x[5:8] + '/' + x[8:12] + '-' + x[12:]
#     return cnpj

In [7]:
df = pd.read_csv('cnae-full-data-730k.csv', low_memory=False, dtype='object')
df = df.drop(columns='NOME FANTASIA').drop_duplicates().copy()

df['CNAE FISCAL SECUNDÁRIA'] = df['CNAE FISCAL SECUNDÁRIA'].apply(lambda x: [] if 'nan' in str(x).split(',') else str(x).split(','))
df.head()

,ID,CNPJ,RAZÃO SOCIAL,NAT. JURIDICA,PORTE DA EMPRESA,INICIO DA ATIVIDADE,CNAE FISCAL PRINCIPAL,CNAE FISCAL SECUNDÁRIA,MUNICÍPIO,UF
0,37066,27459772,JOSE MENDES NETO 10346766796,2135,1.0,2017,4754702,[],5625,ES
1,37067,27459784,IDERLAN ALBERTO SILVA DE SOUZA 01356072135,2135,1.0,2017,4789004,"[5620104, 9609207, 3832700, 4724500, 3831999, ...",9445,GO
2,37068,27459796,T.F.DOS SANTOS MENDES-PRODUTOS DE INFORMATICA ...,2062,1.0,2017,4751201,"[8219901, 4789007, 4753900, 4761003, 4763602, ...",7943,PR
3,37069,27459811,LETICIA LUZ E SILVA 01860757073,2135,1.0,2017,5811500,[3102100],8801,RS
4,37070,27459822,AMIEL PERES DA SILVA 01618944100,2135,1.0,2017,4789099,"[2219600, 9329899, 4783101, 4789001, 8230001, ...",1066,GO


In [8]:
df.to_csv('pre-processed-data-730k.csv', index=False)

In [10]:
df_cliente = pd.read_csv('clientes.csv', sep=';')

In [12]:
df_cliente.shape, df_cliente.drop(columns=['NOME FANTASIA']).drop_duplicates().shape

((101, 11), (101, 10))

In [21]:
df_cliente['CNAE FISCAL SECUNDÁRIA'] = df_cliente['CNAE FISCAL SECUNDÁRIA'].apply(lambda x: [] if 'nan' in str(x).split(',') else str(x).split(','))

In [23]:
df_cliente

,ID,CNPJ,NOME FANTASIA,RAZÃO SOCIAL,NAT. JURIDICA,PORTE DA EMPRESA,INÍCIO DA ATIVIDADE,CNAE FISCAL PRINCIPAL,CNAE FISCAL SECUNDÁRIA,MUNICÍPIO,UF
0,55000001,43016722,ALL SOCKS CWB,ALL SOCKS CWB COMERCIO DE MEIAS LTDA,Sociedade Empresária Limitada,MICRO EMPRESA,2021,4781400,[],CURITIBA,PR
1,55000002,6246280,ATRAENTE LINGERIE LTDA,ATRAENTE LINGERIE LTDA,Sociedade Empresária Limitada,MICRO EMPRESA,2004,4781400,[],CURITIBA,PR
2,55000003,5080448,CHOCOLE,BAYER FLUTUARE LTDA,Sociedade Empresária Limitada,MICRO EMPRESA,2002,4781400,"[4789001, 4713002]",CURITIBA,PR
3,55000004,98514284,LOJA MIRIAM,BELLONY SANTA BROGLIO GEARRAH & CIA LTDA,Sociedade Empresária Limitada,MICRO EMPRESA,1966,4755501,"[4755503, 4755502, 4781400]",VACARIA,RS
4,55000005,34031912,BOT & BOT COMERCIO VAREJISTA DO VESTUARIO LTDA,BOT & BOT COMERCIO VAREJISTA DO VESTUARIO LTDA,Sociedade Empresária Limitada,MICRO EMPRESA,2019,4781400,"[4772500, 4782201, 4782202, 4783101, 4783102, ...",PINHAIS,PR
...,...,...,...,...,...,...,...,...,...,...,...
96,55000097,7272256,TOPA CONFECCOES LTDA,TOPA CONFECCOES LTDA,Sociedade Empresária Limitada,DEMAIS,2005,4781400,[1351100],PATO BRANCO,PR
97,55000098,28017097,INTIMAS LINGERIE,V. BALABUCH NEUMANN - COMERCIO DE ROUPAS,Empresário (Individual),MICRO EMPRESA,2017,4781400,[],CAMPO MOURAO,PR
98,55000099,1324712,VANIA MODAS,V F DO ROSARIO & CIA LTDA,Sociedade Empresária Limitada,MICRO EMPRESA,1996,4781400,[],CASCAVEL,PR
99,55000100,15045507,VK COMERCIO DE CONFECCOES LTDA,VK COMERCIO DE CONFECCOES LTDA,Sociedade Empresária Limitada,DEMAIS,2012,4781400,"[4755503, 4759899, 4772500, 4782201]",CASCAVEL,PR


In [22]:
df_cliente.to_csv('clientes.csv', index=False)